<a href="https://colab.research.google.com/github/kshwan95/Machine-Learning/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#교차 검증과 그리드 서치
import pandas as pd
wine = pd.read_csv('http://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [10]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target= train_test_split(
    data, target, test_size=0.2, random_state=42
)

In [11]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

In [12]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [14]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [15]:
#교차 검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt,train_input, train_target)
print(scores)

{'fit_time': array([0.00893855, 0.00754094, 0.00781417, 0.00816274, 0.00750184]), 'score_time': array([0.00108767, 0.00093341, 0.00096726, 0.00103855, 0.00093198]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [16]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [17]:
#하이퍼파라미터 튜닝
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [19]:
#gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)

In [20]:
gs.fit(train_input,train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [21]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index ])

{'min_impurity_decrease': 0.0001}


In [37]:
params={'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)}

In [38]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [39]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [41]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [42]:
#랜덤서치
from scipy.stats import uniform, randint
rgen = randint(0,10)
rgen.rvs(10)

array([2, 6, 1, 4, 4, 1, 0, 0, 7, 0])

In [44]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([110,  99,  81, 115,  81,  96, 102, 104, 110, 102]))

In [45]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.69406995, 0.20380787, 0.87198091, 0.72124055, 0.45346694,
       0.0017685 , 0.31173452, 0.35004301, 0.09636888, 0.45949266])

In [46]:
params={'min_impurity_decrease':uniform(0.0001,0.001),
        'max_depth':randint(20,50),
        'min_samples_split':randint(2,25),
        'min_samples_leaf':randint(1,25)
        }

In [47]:
from sklearn.model_selection import RandomizedSearchCV
gs =RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params, n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [48]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [49]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [52]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
